In [1]:
# %run MyFunctions.ipynb
# %load_ext autoreload
# %autoreload 2
# import import_ipynb
# import MyFunctions
# from MyFunctions import *

import datetime as dt
import smtplib
import simplejson
import pymssql
import psycopg2
import pandas as pd
# import datetime
from pandasql import sqldf

In [2]:
today = dt.datetime.today().strftime('%m-%d-%Y')

In [3]:
def generate_gh_df_v2():
    
    # generate gh credentials
    with open("/Users/maxwell.lee/OneDrive - Jet/New Folder/Notebooks/Credentials/redshift_creds.json.nogit") as fh:
        creds_gh = simplejson.loads(fh.read())
        
    # generate gh query
    file = open('/Users/maxwell.lee/OneDrive - Jet/New Folder/Queries/candidate_date_details_all_v2.sql', 'r')
    sql_gh = file.read()
    
    #generate column lists
    with open('/Users/maxwell.lee/OneDrive - Jet/New Folder/Queries/master_file_columns.txt') as f:
        columns = f.read().splitlines()
        
    with open('/Users/maxwell.lee/OneDrive - Jet/New Folder/Queries/date_columns.txt') as f:
        date_columns = f.read().splitlines()
        
    with open('/Users/maxwell.lee/OneDrive - Jet/New Folder/Queries/time_columns.txt') as f:
        time_columns = f.read().splitlines()      
        
    # connect to greenhouse
    conn_red = psycopg2.connect(host = creds_gh['host_name'], 
                                port = creds_gh['port_num'], 
                                database = creds_gh['db_name'], 
                                user = creds_gh['user_name'],
                                password = creds_gh['password'])
    
    # open cursor, run the query, fetch results, close cursor, close connection, save results to dataframe
    cur = conn_red.cursor()
    cur.execute(sql_gh)
    results = cur.fetchall()
    cur.close()
    conn_red.close()
    df_gh = pd.DataFrame(results)
    df_gh.columns = columns
    
    # convert datatypes
    for cols in date_columns:
        df_gh[cols] = pd.to_datetime(df_gh[cols], errors = 'coerce').dt.strftime('%m-%d-%Y').replace('NaT', '')
    
    return df_gh

In [3]:
def generate_pdd_df_v2():
    # generate pdd credentials
    with open("/Users/maxwell.lee/OneDrive - Jet/New Folder/Notebooks/Credentials/sqlserver_creds.json.nogit") as fh:
        creds_pdd = simplejson.loads(fh.read())    
        
    # generate pdd query
    file = open('/Users/maxwell.lee/OneDrive - Jet/New Folder/Queries/candidate_date_details_kenexa_v3.sql', 'r')
    sql_pdd = file.read()
    
    #generate column lists
    with open('/Users/maxwell.lee/OneDrive - Jet/New Folder/Queries/master_file_columns.txt') as f:
        columns = f.read().splitlines()
        
    with open('/Users/maxwell.lee/OneDrive - Jet/New Folder/Queries/date_columns.txt') as f:
        date_columns = f.read().splitlines()
        
    with open('/Users/maxwell.lee/OneDrive - Jet/New Folder/Queries/time_columns.txt') as f:
        time_columns = f.read().splitlines()
        
    # connect to pdd
    conn_mssql = pymssql.connect(server = creds_pdd['server'],
                                 user = creds_pdd['user_name'],
                                 password = creds_pdd['password'])
    
    # open cursor, run the query, fetch results, close cursor, close connection, save results to dataframe
    cur = conn_mssql.cursor()
    cur.execute(sql_pdd)
    results = cur.fetchall()
    cur.close()
    conn_mssql.close()
    df_pdd = pd.DataFrame(results)     
    df_pdd.columns = columns

    # convert datatypes
    for cols in date_columns:
        df_pdd[cols] = pd.to_datetime(df_pdd[cols], errors = 'coerce').dt.strftime('%m-%d-%Y').replace('NaT', '')
    
    return df_pdd    

In [5]:
def merge_dataframes(df1, df2, df3):
    
    frames = [df1, df2]
    df = pd.concat(frames)
    df = df.reset_index(drop = True)
    
    df = pd.merge(df, df3, on = 'lookup_key', how = 'left')
    
    return df

In [24]:
def generate_employee_lookup():
    # generate pdd credentials
    with open("/Users/maxwell.lee/OneDrive - Jet/New Folder/Notebooks/Credentials/sqlserver_creds.json.nogit") as fh:
        creds_pdd = simplejson.loads(fh.read())    
        
    # generate pdd query
    file = open('/Users/maxwell.lee/OneDrive - Jet/New Folder/Queries/employee_lookup.sql', 'r')
    sql_pdd = file.read()
    
    columns = ['lookup_key', 'job_effective_date', 'hire_date']
    
    # connect to pdd
    conn_mssql = pymssql.connect(server = creds_pdd['server'],
                                 user = creds_pdd['user_name'],
                                 password = creds_pdd['password'])
    
    # open cursor, run the query, fetch results, close cursor, close connection, save results to dataframe
    cur = conn_mssql.cursor()
    cur.execute(sql_pdd)
    results = cur.fetchall()
    cur.close()
    conn_mssql.close()
    df_pdd = pd.DataFrame(results)     
    df_pdd.columns = columns
    
    df_pdd['job_effective_date'] = pd.to_datetime(df_pdd['job_effective_date'], errors = 'coerce').dt.strftime('%m-%d-%Y').replace('NaT', '')
    df_pdd['hire_date'] = pd.to_datetime(df_pdd['hire_date'], errors = 'coerce').dt.strftime('%m-%d-%Y').replace('NaT', '')
    
    return df_pdd

In [25]:
def generate_fills_report_v2(dataframe, file_date):
    
    # create new df, drop rows and write to excel
    
    # these are the columns used in this report
    with open('/Users/maxwell.lee/OneDrive - Jet/New Folder/Queries/fills_report_columns.txt') as f:
        columns = f.read().splitlines()
    
    df = dataframe
    # drop all candidates with no offer_accepted_date
    df = df[
            (
                (pd.to_datetime(df['anticipated_start_date'], errors = 'coerce') >= dt.datetime.strptime('2019-02-01', '%Y-%m-%d'))
                |
                (pd.to_datetime(df['job_effective_date'], errors = 'coerce') >= dt.datetime.strptime('2019-02-01', '%Y-%m-%d'))
                |
                (pd.to_datetime(df['hire_date'], errors = 'coerce') >= dt.datetime.strptime('2019-02-01', '%Y-%m-%d'))
            )    
            &
            (
                (pd.to_datetime(df['anticipated_start_date'], errors = 'coerce') < dt.datetime.strptime('2020-02-01', '%Y-%m-%d'))
                |
                (pd.to_datetime(df['job_effective_date'], errors = 'coerce') < dt.datetime.strptime('2020-02-01', '%Y-%m-%d'))
                |
                (pd.to_datetime(df['hire_date'], errors = 'coerce') < dt.datetime.strptime('2020-02-01', '%Y-%m-%d'))
            )
            &
            (df['offer_accepted_date'] != '')
            &
            (df['offer_declined_date'] == '')
            & 
            (df['rejected_date'] == '')
           ]
    
    df = df[columns]
    df.to_excel('Fills_report_as_of_%s.xlsx' % file_date, index = False)  

In [6]:
def generate_fills_report(dataframe, file_date):
    
    # create new df, drop rows and write to excel
    
    # these are the columns used in this report
    with open('/Users/maxwell.lee/OneDrive - Jet/New Folder/Queries/fills_report_columns.txt') as f:
        columns = f.read().splitlines()
    
    df = dataframe
    # drop all candidates with no offer_accepted_date
    df = df[
            (pd.to_datetime(df['anticipated_start_date'], errors = 'coerce') >= dt.datetime.strptime('2019-02-01', '%Y-%m-%d'))
  
            &
            (pd.to_datetime(df['anticipated_start_date'], errors = 'coerce') < dt.datetime.strptime('2020-02-01', '%Y-%m-%d'))
            &
            (df['offer_accepted_date'] != '')
            &
            (df['offer_declined_date'] == '')
            & 
            (df['rejected_date'] == '')
           ]
    
    df = df[columns]
    df.to_excel('Custom_fills_report_as_of_%s.xlsx' % file_date, index = False)  

In [8]:
def generate_pending_offers_report(dataframe, file_date):
    
    # create new df, drop rows and write to excel
    
    # these are the columns used in this report
    with open('/Users/maxwell.lee/OneDrive - Jet/New Folder/Queries/fills_report_columns.txt') as f:
        columns = f.read().splitlines()

    df = dataframe
    # drop all candidates with null offer_extended_date
    df = df[
            (pd.to_datetime(df['offer_extended_date'], errors = 'coerce') >= dt.datetime.strptime('2019-02-01', '%Y-%m-%d'))
            &
            (pd.to_datetime(df['offer_extended_date'], errors = 'coerce') < dt.datetime.strptime('2020-02-01', '%Y-%m-%d'))
            &
            (df['offer_accepted_date'] == '')
            &
            (df['rejected_date'] == '')
            &
            (df['offer_declined_date'] == '')
            &
            (
                (df['ta_current_status_mapped'] == 'Offer Extended') | 
                (df['ta_current_status_mapped'] == 'Hired') | 
                (df['ta_current_status_mapped'] == 'Offer Accepted') |
                (df['ta_current_status_mapped'] == 'Selected')
            )
            &
            ((df['job_status_mapped'] == 'Open') | (df['job_status_mapped'] == 'Hold'))
           ]

    df = df[columns]
    df.to_excel('Pending_offers_report_as_of_%s.xlsx' % file_date, index = False)   

In [9]:
def generate_all_offers_report(dataframe, file_date):
    
    # create new df, drop rows and write to excel
    
    # these are the columns used in this report
    with open('/Users/maxwell.lee/OneDrive - Jet/New Folder/Queries/all_offers_report_columns.txt') as f:
        columns = f.read().splitlines()

    df = dataframe
    # drop all candidates with null offer_extended_date
    df = df[
            (pd.to_datetime(df['offer_extended_date'], errors = 'coerce') >= dt.datetime.strptime('2019-02-01', '%Y-%m-%d'))
            &
            (pd.to_datetime(df['offer_extended_date'], errors = 'coerce') < dt.datetime.strptime('2020-02-01', '%Y-%m-%d'))
           ]

    df = df[columns]
    df.to_excel('All_offers_report_as_of_%s.xlsx' % file_date, index = False)   

In [10]:
def generate_opens_report(dataframe, file_date):
    
    # create new df, drop rows and write to excel
    
    # these are the columns used in this report
    with open('/Users/maxwell.lee/OneDrive - Jet/New Folder/Queries/opens_report_columns.txt') as f:
        columns = f.read().splitlines()
        
    file = open('/Users/maxwell.lee/OneDrive - Jet/New Folder/Queries/opens_report.sql', 'r')
    sql_opens_report = file.read()   
        
    sql_df = dataframe[columns]
    
    out = sqldf(sql_opens_report)
    
    out.to_excel('Opens_report_as_of_%s.xlsx' % file_date, index = False)   

In [11]:
def generate_open_funnel_report(dataframe, file_date):
    
    # import columns and sql query for report
    with open('/Users/maxwell.lee/OneDrive - Jet/New Folder/Queries/open_funnel_report_columns.txt') as f:
        columns = f.read().splitlines()
        
    with open('/Users/maxwell.lee/OneDrive - Jet/New Folder/Queries/time_columns.txt') as f:
        time_columns = f.read().splitlines()
        
    file = open('/Users/maxwell.lee/OneDrive - Jet/New Folder/Queries/open_funnels.sql', 'r')
    sql_opens_report = file.read()
    
    # keep columns for report and handle data type issues
    # sql_df = dataframe[columns]
    
    for cols in time_columns:
        dataframe.loc[dataframe[cols].isnull(), cols] = -1
        dataframe[cols] = dataframe[cols].astype(float)
        #dataframe.loc[:, cols] = dataframe[cols].apply(lambda x: float(x))
    
    # generate report and write to excel
    out = sqldf(sql_opens_report)
    out.to_excel('Custom_open_funnels_report_as_of_%s.xlsx' % file_date, index = False)

In [29]:
df_gh = generate_gh_df_v2()

In [15]:
df_pdd = generate_pdd_df_v2()

In [26]:
df_lookup = generate_employee_lookup()

In [27]:
df = merge_dataframes(df_gh, df_pdd, df_lookup)

In [9]:
generate_fills_report(df_pdd, today)

In [10]:
generate_pending_offers_report(df_pdd, today)

NameError: name 'generate_pending_offers_report' is not defined

In [18]:
generate_opens_report(df, today)

In [16]:
generate_open_funnel_report(df_pdd, today)

In [23]:
generate_all_offers_report(df, today)

In [17]:
df_pdd.to_excel('Custom_funnel_raw_%s.xlsx' % today, index = False)